In [1]:
from selenium import webdriver
from selenium.webdriver import Chrome, ChromeOptions
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import  WebDriverWait
import pandas as pd 
import time
import re
import os
import requests

In [67]:
def scarpe(path):
    options = ChromeOptions()
    options.headless=True
    service = Service(ChromeDriverManager().install())
    driver = Chrome(service=service, options=options)
    driver.get(path)
    driver.maximize_window()
    
    # number of cars
    try:
        time.sleep(5)
        cars = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '(.//span[@class="stock-list__results ng-binding"])[1]'))).text.strip()
        if cars:
            print(cars)
            total_cars = int(cars.split(" - ")[1].split(" ")[2])
            # total_cars = int(splited_cars[4])
            print(f"{total_cars} number of cars found")
        else:
            print("No cars found")
    except Exception as e:
        print(f"No cars found and")
    
    
    # result
    results = []
    car_count = 0
    visited_pages = set()
    while car_count < 2: #total_cars
        
        try:
            page_cars = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH, './/div[@class="car__picture"]')))
            
            for i in range(2): # len(page_cars)
                if car_count >= total_cars:
                    break
                
                try:
                    page_cars = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH, './/div[@class="car__picture"]')))
                    driver.execute_script("arguments[0].scrollIntoView();", page_cars[i])
                    time.sleep(1)
                    page_cars[i].click()

                    details = {}    
                    try:
                        det_card = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, './/div[@class="vehicle__details"]')))
                        if det_card:
                            # title --> done
                            try:
                                titlle = WebDriverWait(det_card, 5).until(EC.presence_of_element_located((By.XPATH, './/div[@class="vehicle__details-top"]'))).text.strip()
                                if titlle:
                                    details['Title'] = titlle
                                else:
                                    details['Title'] = "na"
                            except Exception as e:
                                print("No car title found")
                        
                            
                            # other information --> not done
                            try:
                                dets_card =WebDriverWait(det_card, 10).until(EC.presence_of_element_located((By.XPATH, '(.//div[@ng-hide="Listing.IsPlantItem"])[2]'))) 
                                if dets_card:
                                    dets = WebDriverWait(dets_card, 5).until(EC.presence_of_all_elements_located((By.XPATH, './/div[@class="vehicle__details-single"]'))) 
                                    for det in dets:
                                        info_lbl = det.find_element(By.XPATH, './/span[contains(@class, "vehicle__details-single-item-title") or (contains(@class, "vehicle__details-single-item-title ng-binding"))]').text.strip()
                                        info_val = det.find_element(By.XPATH, './/span[@class="vehicle__details-single-item-value ng-binding"]').text.strip()
                                        if info_lbl and info_val:
                                            details[info_lbl] = info_val
                                else:
                                    print("No information")
                            except Exception as e:
                                print(f"No information {e}")
                    except Exception as e:
                        print("No det card found") 
                    
                    # images
                    try:
                        imgs= WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH, './/img[@class="ng-scope ug-thumb-image"]')))
                        if imgs:
                            imgs_lst = [img.get_attribute("src") for img in imgs]
                            details['Images'] = ", ".join(imgs_lst)
                        else:
                            details['Images'] = "na"
                    except Exception as e:
                        print("No images found")
                        
                    
                    car_count+=1
                    results.append(details)
                    driver.back()
                
                except Exception as e:
                    print("No more cars found")
                    
            
            if car_count % 30 == 0: # --> 25%25 == 0 and 50%25== 0 --> issue here
                try:
                    next_card =WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '(.//div[@class="stock-list__pages"])[2]'))) 
                    next_link = WebDriverWait(next_card, 10).until(EC.presence_of_all_elements_located((By.XPATH, './/a[contains(@class,"ng-scope button-dot") and not (contains(@class, "ng-scope button-dot active"))]')))
                    if next_link:
                        for nxt_btn in next_link:
                            page_url = nxt_btn.get_attribute("href")
                            if page_url and page_url not in visited_pages:  # Check if the page has not been visited
                                # print(f"Visiting: {page_url}")
                                visited_pages.add(page_url)  # Mark it as visited
                                nxt_btn.click()
                        
                                WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH, './/div[@class="car__picture"]')))
                                print("Hit the next button")
                                break
                except Exception as e:
                    print("No next found")
                    break 
        except Exception as e:
            print("No page cars found")
            break
            
    
    df = pd.DataFrame.from_dict(results)
    df.to_csv("lca.csv", index=False)
    
    time.sleep(2)
    driver.quit()
path = "https://www.leominstercarauctions.co.uk/stock-list?saleid=261"
scarpe(path)

SHOWING 1 - 24 OF 85 RESULTS
85 number of cars found


In [71]:
df = pd.read_csv("lca.csv")
df

,Title,Lot Number:,Reg Number:,Registered:,Mileage:,Fuel Type:,Transmission:,Body:,Colour:,V5:,MOT:,Warranty:,VAT Status:,Images,Former Keepers:
0,FORD TRANSIT\n85 T260S FWD - 2198cc,400,MF56PGK,Sep 2006,"172,180 Not Warranted",Diesel,Manual,Van,White,V5 with auction,07/05/2025,Sold as Seen,NaN,https://www.leominstercarauctions.co.uk/image....,NaN
1,RENAULT MASTER\n(X70) - 2463cc,401,WR03UEG,Aug 2003,NaN,Diesel,Manual,Minibus,White,V5 with auction,NaN,Sold as Seen,NaN,https://www.leominstercarauctions.co.uk/image....,1.0


In [72]:
from urllib.parse import urlparse, urljoin
# urlparse: Parses a URL into components (scheme, netloc, path, etc.), making it easy to validate or extract information from the URL.
# urljoin: Joins a base URL with a relative path to form a complete URL.

reg_img = df[["Reg Number:", "Images"]]

def download_images(data, main_folder="Images"): 
    os.makedirs(main_folder, exist_ok=True)
    
    for index, row in data.iterrows():
        reg_no = row["Reg Number:"]
        image_urls = row["Images"]

        # Check if 'Images' column is empty or NaN
        if pd.isna(image_urls) or not isinstance(image_urls, str) or image_urls.strip() == "":
            print(f"Skipping {reg_no} (No image URLs)")
            continue

        image_urls = image_urls.split(", ")  # Split URLs by comma
        
        reg_folder = os.path.join(main_folder, reg_no)
        os.makedirs(reg_folder, exist_ok=True)
        
        for idx, url in enumerate(image_urls):
            url = url.strip()
            if not url.startswith(("http://", "https://")):
                url = urljoin("https://", url) 
            
            parsed_url = urlparse(url)
            if not parsed_url.scheme or not parsed_url.netloc:
                print(f"Invalid URL skipped: {url}") 
                continue
            
            try:
                response = requests.get(url, stream=True) 
                response.raise_for_status()
                
                file_name = os.path.basename(parsed_url.path) or f"image_{idx + 1}.jpg"
                file_extension = file_name.split(".")[-1]
                
                if file_extension not in ["jpg", "jpeg", "png", "gif", "bmp", "webp"]:
                    file_name = f"image_{idx + 1}.jpg"
                
                full_file_name = os.path.join(reg_folder, f"{reg_no}_{idx + 1}_{file_name}")
                
                with open(full_file_name, 'wb') as f:
                    for chunk in response.iter_content(1024):
                        f.write(chunk)
                
                print(f"Downloaded: {full_file_name}")
            except Exception as e:
                print(f"Failed to download {url} for {reg_no}: {e}")

# Call the function
download_images(reg_img)

Downloaded: Images\MF56PGK\MF56PGK_1_image_1.jpg
Downloaded: Images\MF56PGK\MF56PGK_2_image_2.jpg
Downloaded: Images\MF56PGK\MF56PGK_3_image_3.jpg
Downloaded: Images\MF56PGK\MF56PGK_4_image_4.jpg
Downloaded: Images\MF56PGK\MF56PGK_5_image_5.jpg
Downloaded: Images\MF56PGK\MF56PGK_6_image_6.jpg
Downloaded: Images\MF56PGK\MF56PGK_7_image_7.jpg
Downloaded: Images\MF56PGK\MF56PGK_8_image_8.jpg
Downloaded: Images\WR03UEG\WR03UEG_1_image_1.jpg
Downloaded: Images\WR03UEG\WR03UEG_2_image_2.jpg
Downloaded: Images\WR03UEG\WR03UEG_3_image_3.jpg
Downloaded: Images\WR03UEG\WR03UEG_4_image_4.jpg
Downloaded: Images\WR03UEG\WR03UEG_5_image_5.jpg
Downloaded: Images\WR03UEG\WR03UEG_6_image_6.jpg
Downloaded: Images\WR03UEG\WR03UEG_7_image_7.jpg
